# Liquify

In [182]:
# General
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import math

# Remote Sensing/Geo
import rasterio as rio
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em

### Preprocessing

In [179]:
class preprocessing():
    def __init__(self,data,MTL):
        '''
        Perform preprocessing steps (TOA radiometric and atmospheric coreections) on a single band of data.
        
        Parameters
        ----------
        data: array-like
            2d data (image) of a single Landsat band
        MTL: str
            MTL file name for given data
        '''
        self.data = data
        self.mtl = MTL
    def TOA_radiometric_corr(self,band):
        '''
        Top of atmosphere radiometric correction using gain and bias method.
        
        Parameter
        ---------
        band: int
            Landsat 8 band number (in order i.e. 2 = blue band)
        '''
        # Step 0: Load MTL to extract needed values
        df = gpd.read_file(self.mtl)
        self.df = df
        
        # Step 1: DN to radiance
        g = df.RADIANCE_MULT
        b = df.RADIANCE_ADD
    
        rad_arr = (self.data * g) + b
        
        # Step 2: Radiance to TOA reflectance
        ESUN = [0.,0.,2067.,1893.,1603.,9726.,245.,79.72,0.,399.7]
        ESUN = ESUN[band]
        
        d = df.EARTH_SUN_DISTANCE
        phi = 90 - df.SUN_ELEVATION

        ref_arr = (np.pi * rad_arr * d**2) / (ESUN * math.cos(phi*math.pi/180))
        self.toa_data = ref_arr
        return self.toa_data
    def atmospheric_corr(self):
        '''
        Atmospheric correction using dark subtraction method. Converts TOA reflectance to surface reflectance.
        '''
        toa_data = np.array(self.toa_data)
        self.atm_data = self.toa_data - np.min(toa_data)
        return self.atm_data

### Masking

In [181]:
class masking():
    def __init__(self,data,mask,shape_file=None):
        '''
        Apply a cloud mask and optional boundary mask to image.
        
        Parameters
        ----------
        data: array-like
            2d array of Landsat 8 image
        mask:
            landsat_qa cloud file
        shape_file: obj
            object of loaded .shp file
        '''
        self.data = data
        self.mask = mask
        self.shape_file = shape_file
    def cloud_val(self):
        '''
        Calculate masking values for a cloud mask of the given dataset.
        '''        
        cloud = em.pixel_flags['pixel_qa']['L8']['Cloud']
        cloud_shadow = em.pixel_flags['pixel_qa']['L8']['Cloud Shadow']
        high_confidence_cloud = em.pixel_flags['pixel_qa']['L8']['High Cloud Confidence']

        self.masked_val = cloud_shadow + cloud + high_confidence_cloud
        return self.masked_val
    def cloud_mask(self):
        '''
        Using mask file and your calculated masking values to apply a cloud mask to your image.
        '''
        self.masked_data = em.mask_pixels(self.data, self.mask, vals=self.masked_val)
        return self.masked_data
    def load_shape_file():
        '''
        Load a polygon shape file (.shp) as a geopandas df to mask outside of a specific boundary.
        '''
        if self.shape_file != None:
            self.boundary = gpd.read_file(self.shape_file)
            return self.boundary
    def boundary_mask(self):
        '''
        Mask pixels of data outside a given shape file.
        '''
        if self.shape_file != None:
            with rio.open(self.data) as src:
                self.raster, self.meta = es.crop_image(src,self.boundary)
            return self.crop

### Water Classification

In [184]:
class water_class():
    def __init__(self,data):
        self.data = data
    def identify_water(self):
        self.water_val = em.pixel_flags['pixel_qa']['L8']['Water'] # Pulling Landsat 8 water values
        return fig, ax
    def water_plot(self,tick_loc,title=None,figsize=(15,13),cmap=plt.cm.get_cmap('Blues', 2)):
        '''
        Plot with labeled water and non-water regions, setting default options and easy tweaking of parameters
        
        Parameters
        ----------
        tick_loc: int/float
            where on colorbar to label 'Other'(not water) based on data values of image
        title: str
            title of plot
        figsize: tuple, optional
            figure size to use. Default: (15,13)
        cmap: str, optional
            Colormap to use for the image. Default: 'Greys_r'
        
        Returns
        -------
        fig, ax
            figure and axes objects containing currently plotted data.
        '''
        fig, ax = plt.subplots(figsize=figsize)

        im = ax.imshow(self.data,cmap=cmap)

        cbar = ep.colorbar(im)
        cbar.set_ticks((tick_loc,self.water_value[0]))
        cbar.ax.set_yticklabels(['Other', 'Water'])
        
        if title != None:
            ax.set_title(title,fontsize=15)
    
        ax.set_axis_off()
        return fig, ax